In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [4]:
# # Load the train and test sets
# train_df = pd.read_csv('train_data.csv')
# test_df = pd.read_csv('test_data.csv')

# # Combine the two datasets. Set ignore_index to False.
# df = pd.concat([train_df,test_df],ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)
# df.to_csv('data.csv',index=False)

In [2]:
df=pd.read_csv("backup.csv")
# df =df.sample(10000,random_state=42)
# df.to_csv('sample.csv',index=False)
df.head()

,title,link,domain,published_date,topic
0,Waikato Times: Hamilton and Waikato news,https://www.stuff.co.nz/waikato-times,stuff.co.nz,2012-09-16 04:44:50,ENTERTAINMENT
1,Southland Times: Southland /Invercargill news,https://www.stuff.co.nz/southland-times,stuff.co.nz,2012-09-16 04:44:57,SPORTS
2,"Sports: Rugby, cricket, football, All Blacks",https://www.stuff.co.nz/sport,stuff.co.nz,2012-09-16 04:46:00,SPORTS
3,Real-Time News from PennLive,https://www.pennlive.com/news/,pennlive.com,2015-10-23 16:44:48,SCIENCE
4,2020 Playoffs: Feels like 2010…,https://www.nhl.com/canadiens/news/,nhl.com,2016-09-23 08:46:28,SPORTS


In [3]:
df=df.drop(["link","domain","published_date"],axis=1)

In [4]:
#The percentage of each class in the data

df.topic.value_counts(normalize=True).sort_index().mul(100).round(1).astype(str) + '%'

topic
BUSINESS         13.8%
ENTERTAINMENT    13.8%
HEALTH           13.8%
NATION           13.8%
SCIENCE           3.5%
SPORTS           13.8%
TECHNOLOGY       13.8%
WORLD            13.8%
Name: proportion, dtype: object

In [5]:
# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['title'] = df['title'].apply(lower_case)
        df['title'] = df['title'].apply(remove_stop_words)
        df['title'] = df['title'].apply(removing_numbers)
        df['title'] = df['title'].apply(removing_punctuations)
        df['title'] = df['title'].apply(removing_urls)
        df['title'] = df['title'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [6]:
df=normalize_text(df)
df.head()

,title,topic
0,waikato time hamilton waikato news,ENTERTAINMENT
1,southland time southland invercargill news,SPORTS
2,sport rugby cricket football black,SPORTS
3,real time news pennlive,SCIENCE
4,playoff feel like …,SPORTS


In [12]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df["topic"] = label_encoder.fit_transform(df["topic"])

In [13]:
vectorizer = TfidfVectorizer(max_features=20000)
X = vectorizer.fit_transform(df['title'])
y = df['topic']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/bhauryal7/NewsClassification.mlflow')
dagshub.init(repo_owner='bhauryal7', repo_name='NewsClassification', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=0220acbd-dd69-4c26-b2bf-5f7115d09138&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=5c8677e544562eeae0083c2259f4e78360c44e15ce463689c49992895013c498




Accessing as bhauryal7

Initialized MLflow to track repo "bhauryal7/NewsClassification"

Repository bhauryal7/NewsClassification initialized!

<Experiment: artifact_location='mlflow-artifacts:/49127cd8129d4f7da5c8e6ddcf808605', creation_time=1742312611100, experiment_id='0', last_update_time=1742312611100, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [15]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "TfidfVectorizer")
        mlflow.log_param("num_features", 20000)
        mlflow.log_param("test_size", 0.2)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(C=1,max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)

        class_report = classification_report(y_test,y_pred)

        print("\nClassification Report:\n", class_report)

        # Log metrics in MLflow
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_text(class_report, "classification_report.txt")

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-04-03 16:11:10,990 - INFO - Starting MLflow run...
2025-04-03 16:11:11,221 - INFO - Logging preprocessing parameters...
2025-04-03 16:11:11,932 - INFO - Initializing Logistic Regression model...
2025-04-03 16:11:11,933 - INFO - Fitting the model...
2025-04-03 16:11:17,079 - INFO - Model training complete.
2025-04-03 16:11:17,079 - INFO - Logging model parameters...
2025-04-03 16:11:17,296 - INFO - Making predictions...
2025-04-03 16:11:17,309 - INFO - Calculating evaluation metrics...



Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.79      0.79      2931
           1       0.84      0.89      0.86      3093
           2       0.77      0.83      0.80      3029
           3       0.65      0.70      0.68      2984
           4       0.89      0.76      0.82       713
           5       0.92      0.92      0.92      2932
           6       0.90      0.86      0.88      3021
           7       0.72      0.63      0.67      3052

    accuracy                           0.80     21755
   macro avg       0.81      0.80      0.80     21755
weighted avg       0.80      0.80      0.80     21755



2025-04-03 16:11:18,346 - INFO - Saving and logging the model...
2025/04/03 16:11:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-04-03 16:11:27,668 - INFO - Model training and logging completed in 16.45 seconds.


🏃 View run bald-boar-700 at: https://dagshub.com/bhauryal7/NewsClassification.mlflow/#/experiments/0/runs/1db01c1c34dd4ef5aed16ec7207927dd
🧪 View experiment at: https://dagshub.com/bhauryal7/NewsClassification.mlflow/#/experiments/0
